In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [2]:
data = pd.read_excel('brazilian credit card.xlsx')
selected_vars =['cep_cat numeric',
 'score_cat numeric',
 'velocidade_cat numeric',
 'bandeira_cat numeric',
 'qtde_parc_cat numeric',
 'valor_trans_cat numeric',
 'trans_nacional_cat numeric',
 'mcc_cat numeric',
 'valor_trans_ant_cat numeric',
 'mcc_ant_cat numeric',
 'dif_score_cat numeric',
 'cep_ant_cat numeric',
 'lim_cred_cat numeric',
 'Class']

In [3]:
tr_data = data[selected_vars]
tr_data.shape

(374823, 14)

In [4]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [5]:
y=tr_data['Class']

In [6]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
X=pd.DataFrame(StandardScaler().fit_transform(X))

In [7]:
X_train0,X_tset,y_train0,y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=0)
X_train0,X_train2,y_train0,y_train2 = train_test_split(X_train0, y_train0, test_size=0.333, shuffle=False,random_state=0)
X_train0,X_train1,y_train0,y_train1 = train_test_split(X_train0, y_train0, test_size=0.5, shuffle=False,random_state=0)

from sklearn.linear_model import SGDOneClassSVM
df1_normal=X_train0[y_train0==0]
clf = SGDOneClassSVM(nu=0.013,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train0.drop(X_train0.index[index], inplace=True)
y_train0.drop(y_train0.index[index], inplace=True)

df1_normal=X_train1[y_train1==0]
clf = SGDOneClassSVM(nu=0.014,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train1.drop(X_train1.index[index], inplace=True)
y_train1.drop(y_train1.index[index], inplace=True)

df1_normal=X_train2[y_train2==0]
clf = SGDOneClassSVM(nu=0.014,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train2.drop(X_train2.index[index], inplace=True)
y_train2.drop(y_train2.index[index], inplace=True)

In [8]:
X_train0.shape,X_train1.shape

((82207, 13), (86807, 13))

In [9]:
y_train0.shape,y_train1.shape

((82207,), (86807,))

In [10]:
X_tset.shape

(112447, 13)

# N=1

In [11]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X0=X_train0.to_numpy()
test_LSTM_X=X_tset.to_numpy()

## Reshape input to be 3D [samples, timesteps, features] (format requis par LSTM)
train_LSTM_X0 = train_LSTM_X0.reshape((train_LSTM_X0.shape[0], 1, train_LSTM_X0.shape[1]))
test_LSTM_X = test_LSTM_X.reshape((test_LSTM_X.shape[0], 1, test_LSTM_X.shape[1]))

## Recuperation des labels
train_LSTM_y0=y_train0
test_LSTM_y=y_test

In [12]:
inputs=Input((1,13))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model0=Model(inputs,outputs)

In [13]:
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [14]:
history0=model0.fit(train_LSTM_X0, train_LSTM_y0,epochs=500,batch_size=200)

Epoch 1/500
412/412 [==============================] - 4s 4ms/step - loss: 0.2529 - precision: 0.1334 - recall: 0.0890
Epoch 2/500
412/412 [==============================] - 1s 3ms/step - loss: 0.1167 - precision: 0.5446 - recall: 0.0784
Epoch 3/500
412/412 [==============================] - 1s 4ms/step - loss: 0.1129 - precision: 0.5549 - recall: 0.0933
Epoch 4/500
412/412 [==============================] - 1s 3ms/step - loss: 0.1107 - precision: 0.5663 - recall: 0.0985
Epoch 5/500
412/412 [==============================] - 1s 3ms/step - loss: 0.1090 - precision: 0.5856 - recall: 0.1048
Epoch 6/500
412/412 [==============================] - 1s 3ms/step - loss: 0.1076 - precision: 0.5912 - recall: 0.1068
Epoch 7/500
412/412 [==============================] - 1s 3ms/step - loss: 0.1063 - precision: 0.5953 - recall: 0.1173
Epoch 8/500
412/412 [==============================] - 1s 3ms/step - loss: 0.1052 - precision: 0.5985 - recall: 0.1282
Epoch 9/500
412/412 [===========================

In [15]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]> 0.5

In [16]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs0)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs0)
print('f1: %f' % f1)

Accuracy: 0.994691
Precision: 0.950679
Recall: 0.904044
f1: 0.926775


# N=2

In [17]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X1=X_train1.to_numpy()
## Reshape input to be 3D [samples, timesteps, features] (format requis par LSTM)
train_LSTM_X1 = train_LSTM_X1.reshape((train_LSTM_X1.shape[0], 1, train_LSTM_X1.shape[1]))
## Recuperation des labels
train_LSTM_y1=y_train1

In [18]:
inputs=Input((1,13))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs,outputs)

In [19]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [20]:
history1=model1.fit(train_LSTM_X1, train_LSTM_y1,epochs=500,batch_size=200)

Epoch 1/500
435/435 [==============================] - 4s 3ms/step - loss: 0.2485 - precision: 0.1821 - recall: 0.1369
Epoch 2/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1178 - precision: 0.5106 - recall: 0.0959
Epoch 3/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1142 - precision: 0.5480 - recall: 0.0889
Epoch 4/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1122 - precision: 0.5709 - recall: 0.1033
Epoch 5/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1104 - precision: 0.5739 - recall: 0.1033
Epoch 6/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1088 - precision: 0.6053 - recall: 0.1115
Epoch 7/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1074 - precision: 0.6273 - recall: 0.1250
Epoch 8/500
435/435 [==============================] - 1s 3ms/step - loss: 0.1060 - precision: 0.6307 - recall: 0.1231
Epoch 9/500
435/435 [===========================

In [21]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]> 0.5

In [22]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs1)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs1)
print('f1: %f' % f1)

Accuracy: 0.994726
Precision: 0.959037
Recall: 0.896387
f1: 0.926654


# N=3

In [23]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X2=X_train2.to_numpy()
## Reshape input to be 3D [samples, timesteps, features] (format requis par LSTM)
train_LSTM_X2 = train_LSTM_X2.reshape((train_LSTM_X2.shape[0], 1, train_LSTM_X2.shape[1]))
## Recuperation des labels
train_LSTM_y2=y_train2

In [24]:
inputs=Input((1,13))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model2=Model(inputs,outputs)

In [25]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [26]:
history2=model2.fit(train_LSTM_X2, train_LSTM_y2,epochs=500,batch_size=200)

Epoch 1/500
437/437 [==============================] - 4s 4ms/step - loss: 0.2464 - precision: 0.0978 - recall: 0.0573
Epoch 2/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1177 - precision: 0.5222 - recall: 0.0928
Epoch 3/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1141 - precision: 0.5691 - recall: 0.1038
Epoch 4/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1119 - precision: 0.5883 - recall: 0.1041
Epoch 5/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1101 - precision: 0.5893 - recall: 0.1171
Epoch 6/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1084 - precision: 0.6082 - recall: 0.1211
Epoch 7/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1071 - precision: 0.5947 - recall: 0.1305
Epoch 8/500
437/437 [==============================] - 2s 4ms/step - loss: 0.1057 - precision: 0.6063 - recall: 0.1350
Epoch 9/500
437/437 [===========================

In [27]:
# predict probabilities for test set
yhat_probs2 = model2.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs2 = yhat_probs2[:, 0]> 0.5

In [28]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs2)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs2)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs2)
print('f1: %f' % f1)

Accuracy: 0.994824
Precision: 0.962934
Recall: 0.895190
f1: 0.927827


# FFNN

In [29]:
y_probs0 = model0.predict(train_LSTM_X0, verbose=0)
y_probs1 = model1.predict(train_LSTM_X0, verbose=0)
y_probs2 = model2.predict(train_LSTM_X0, verbose=0)
y_probs00 = model0.predict(train_LSTM_X1, verbose=0)
y_probs11 = model1.predict(train_LSTM_X1, verbose=0)
y_probs22 = model2.predict(train_LSTM_X1, verbose=0)

In [30]:
X_train0=np.concatenate((y_probs0,y_probs1,y_probs2),axis=1)
X_train1=np.concatenate((y_probs00,y_probs11,y_probs22),axis=1)
X_train=np.concatenate((X_train0,X_train1),axis=0)
y_train=np.concatenate((y_train0,y_train1),axis=0)
X_train.shape,y_train.shape

((169014, 3), (169014,))

In [31]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_train, y_train)

# evaluation

In [32]:
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
yhat_probs1 = model1.predict(test_LSTM_X,verbose=0)
yhat_probs2 = model2.predict(test_LSTM_X,verbose=0)
X_test=np.concatenate((yhat_probs0,yhat_probs1,yhat_probs2),axis=1)
y_pred=clf.predict(X_test)

In [33]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_LSTM_y, y_pred)
print('f1: %f' % f1)

Accuracy: 0.997288
Precision: 0.981841
Recall: 0.944484
f1: 0.962800
